In [3]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [115]:
class Module(object):
    """
    Basically, you can think of a module as of a something (black box)
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`:

        output = module.forward(input)

    The module should be able to perform a backward pass: to differentiate the `forward` function.
    Moreover, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule.

        input_grad = module.backward(input, output_grad)
    """

    def __init__(self):
        self._output = None
        self._input_grad = None
        self.training = True

    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        self._output = self._compute_output(input)
        return self._output

    def backward(self, input, output_grad):
        """
        Performs a backpropagation step through the module, with respect to the given input.

        This includes
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self._input_grad = self._compute_input_grad(input, output_grad)
        self._update_parameters_grad(input, output_grad)
        return self._input_grad

    def _compute_output(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which will be stored in the `_output` field.

        Example: in case of identity operation:

        output = input
        return output
        """
        raise NotImplementedError

    def _compute_input_grad(self, input, output_grad):
        """
        Returns the gradient of the module with respect to its own input.
        The shape of the returned value is always the same as the shape of `input`.

        Example: in case of identity operation:
        input_grad = output_grad
        return input_grad
        """

        raise NotImplementedError

    def _update_parameters_grad(self, input, output_grad):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass

    def zero_grad(self):
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass

    def get_parameters(self):
        """
        Returns a list with its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def get_parameters_grad(self):
        """
        Returns a list with gradients with respect to its parameters.
        If the module does not have parameters return empty list.
        """
        return []

    def train(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True

    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Module"

# Layers

## 1. Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ — mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance: 
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance. 

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

----

Let's walk through the process of backpropagation. Given a composition of functions 
$L_n \equiv L \circ f_1 \circ \dots \circ f_n$, 
$f_i : \mathbb{R}^{d_{i}} \rightarrow \mathbb{R}^{d_{i - 1}},
L :  \mathbb{R}^{d_0} \rightarrow \mathbb{R}$ and a point $x_n \in \mathbb{R}^{d_n}$, we want to find the update value $\Delta x_n$, such that composition value decreases: $L_n(x_n + \Delta x_n) < L_n(x_n)$. Notice that in this loose formulation, the task isn't fully determined. The natural way to make it determined is to place a restriction on the length of $\Delta x_n$ and ask for largest loss decrease. For now let's keep the freedom to choose the exact nature of the restriction.

Suppose we know the value of gradient $\nabla L_n$ at point $x_n$. Then $\Delta L \equiv L_n(x_n+\Delta x_n) - L_n(x_n) \approx \nabla L_n(x_n)\Delta x_n$. If we assume this approximation to be an equality, then $\Delta x = -\nabla L_n(x) \epsilon$ will be the vector that leads to largest (approximate) loss decrease among vectors with same length. This kind of restriction is based mostly on computational convenience, and will affect the loss in a complex way: $\Delta L \approx -\nabla L_n^2(x_n) \epsilon$. Considering that in practice all vectors get updated simultaneously, more realistic approximation is this: $\Delta L \approx -\epsilon\sum_n\nabla L_n^2(x_n)$. The quadratic dependence on gradient values can make the gradient descent unstable in some cases and too slow in others, and often requires some heuristics to improve it's performance, like gradient clipping and momentum. Also it's intuitive to choose $\epsilon = \frac{1}{|\nabla L_n|}$ to achieve more stable linear gradient descent, which is basically what the Adam learning rate scheduler does.

Ok, but how do we find $\nabla L_n(x_n)$ for each n? Let's use the chain rule: 

$$\nabla L_{n}(x_{n}) = \nabla (L_{n-1} \circ f_n) (x_{n}) = J_n^T(x_n) \cdot \nabla L_{n-1}(f_{n}(x_n))=
J_n^T(x_n) \cdot \nabla L_{n-1}(x_{n-1}):$$

$$ \nabla (L_{n-1} \circ f_n)(x_n)_i = 
\cfrac{\partial (L_{n-1} \circ f_n)}{\partial x_n^i}(x_n) = 
\sum_{j\in 1\dots d_{n-1}} \cfrac{\partial L_{n-1}}{\partial x_{n-1}^j}(f_n(x_n)) \cfrac{\partial f_n^j}{\partial x_n^i}(x_n)=
\sum_{j\in 1\dots d_{n-1}} \nabla L_{n-1}(f_{n}(x_n)) J_n^{ji}(x_n)
$$


So, knowing $\nabla L_{n-1}(x_{n-1})$ and jacobian $J_n$ at point $x_n$, we can calculate $\nabla L_{n}(x_{n})$. To complete the induction, we need its base and a way to calculate $J_n(x_n)$. Let's assign fictive $\nabla L_{-1}(x_{-1}) = 1$, then $\nabla L_{0}(x_{0}) = J_0^T(x_0) \cdot \nabla L_{-1}(x_{-1}) = J_0^T(x_0) = \nabla L(x_0)$. So, everything came down to jacobians. To find the points $x_n$ at which to calculate the jacobians, we'll need to do a so-called forward pass through the composition chain. Then, using functions with closed-form partial derivatives, or a lookup table, calculate the jacobians. Here we can notice that technically we don't need to know the loss value $L(x_0)$, but only its gradient at any point. This can be usefull when we know the gradient and don't want to solve the differential equation to find the expression for the loss function (which may not even be possible), for example in recommending tasks where we can compare two objects, but not assign an absolute value to each one.

Batchnorm output:

$$f: \mathbb{R}^{B\times N}\rightarrow \mathbb{R}^{B\times N},
f(x) = \cfrac{x - \mu(x)}{\sqrt{\text{Var}(x) + \epsilon}},\,
f(x)_{bn} = \cfrac{x_{bn} - \mu(x)_n}{\sqrt{\text{Var}(x)_n + \epsilon}},$$

$$\mu(x)_{n} = \frac{\sum_i x_{in}}{B},\\
\cfrac{\partial \mu}{\partial x_{b_1n_1}}(x)_{n_0} = \cfrac{[n_0 == n_1]}{B}$$

$$
\text{Var}(x)_{n} = \frac{\sum_i (x_{in} - \mu(x)_{n})^2}{B},\\
\cfrac{\partial \text{Var}}{\partial x_{b_1n_1}}(x)_{n_0} = 
\cfrac{\sum_i 2\bigl([i == b_1 \land n_0 == n_1] - \cfrac{[n_0 == n_1]}{B}\bigr)
\bigl(x_{i n_0} - \mu(x)_{n_0} \bigr)}{B}= \\
 = \cfrac{2[n_0 == n_1]}{B}
\bigl(x_{b_1n_0} - \mu(x)_{n_0} \bigr) +
\cfrac{2[ n_0 == n_1]}{B^2}
\bigl( B \mu(x)_{n_0}  - \sum_i x_{i n_0} \bigr) = 
\cfrac{2[ n_0 == n_1]}{B}
\bigl(x_{b_1n_0} - \mu(x)_{n_0} \bigr)
$$

Let $\nabla \hat L$ be the output gradient, then batchnorm gradient $\nabla L$ is expressed in this way:
 
$$ \nabla L(x)_{ij} = 
\sum_{b \in 1\dots B, \, n\in 1\dots N} \nabla \hat L_{bn}(f(x)) \cfrac{\partial f_{bn}}{\partial x_{ij}}(x)=
$$

$$
= \sum_{b, \, n}
\cfrac{\bigl(
\cfrac{\partial x_{b n}}{\partial x_{i j}}(x) - \cfrac{\partial \mu}{\partial x_{ij}}(x)_{ n}
\bigr)\sqrt{\text{Var}(x)_{n} + \epsilon} - 
(x_{bn} - \mu(x)_{ n})\cfrac{1}{2\sqrt{\text{Var}(x)_{n} + \epsilon}}\cfrac{\partial \text{Var}}{\partial x_{ij}}(x)_{n}}
{\text{Var}(x)_{n} + \epsilon}
\nabla \hat L_{bn} = \\
= \sum_{b, \, n}
\cfrac{\bigl([b == i \land n == j] - [n == j]\cfrac{1}{B}\bigr)(\text{Var}(x)_{n} + \epsilon) - (x_{b n} - \mu(x)_{ n})\cfrac{[n == j]}{B}
\bigl(x_{i n} - \mu(x)_{n} \bigr)}
{\sqrt{\text{Var}(x)_{n} + \epsilon}^3}
\nabla \hat L_{bn} = \\
= \sum_{b}
\cfrac{[i == b] - \cfrac{1}{B} }
{\sqrt{\text{Var}(x)_{j} + \epsilon}}
\nabla \hat L_{bj} -
\cfrac{(x_{i j} - \mu(x)_{ j})
 }
{B\sqrt{\text{Var}(x)_{j} + \epsilon}^3}
\sum_{b}(x_{b j} - \mu(x)_{j})
\nabla \hat L_{bj}
$$

Let:

$\text{output_grad} = \nabla \hat L\\
\text{inverse_std} = \cfrac{1}{\sqrt{\text{Var}(x) + \epsilon}}\\
\text{eye_minus_inverse_b} = E - \cfrac{1}{B}\\
\text{centered_input} = x - \mu(x)
$

Then:

$$\text{grad_input}_{ij} =
\sum_b \text{eye_minus_inverse_b}_{ib} \cdot \text{inverse_std}_{j} \cdot \text{output_grad}_{bj} +\\
-\cfrac{1}{B}\sum_b \text{centered_input}_{ij} \cdot\text{centered_input}_{bj} \cdot \text{inverse_std}^3_{j} \cdot \text{output_grad}_{bj} := \text{first} -\cfrac{1}{B}\text{second}
$$

In [118]:
class BatchNormalization(Module):
    EPS = 1e-3

    def __init__(self, alpha=0.0):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = 0.0
        self.moving_variance = 1.0

    def _compute_output(self, input):
        if self.training == False:
            output = (input - self.moving_mean) / (
                self.moving_variance + self.EPS
            ) ** 0.5
            return output

        mean = input.mean(axis=0)
        var = input.var(axis=0)
        output = (input - mean) / (var + self.EPS) ** 0.5

        self.moving_mean = self.moving_mean * self.alpha + mean * (1 - self.alpha)
        self.moving_variance = self.moving_variance * self.alpha + var * (
            1 - self.alpha
        )

        return output

    def _compute_input_grad(self, input, output_grad):
        inverse_std = (input.var(axis=0) + self.EPS) ** (-1 / 2)
        B, N = input.shape
        eye_minus_inverse_b = np.eye(B) - 1 / B
        centered_input = input - input.mean(axis=0)

        first = np.einsum(
            "ib, j, bj -> ij", eye_minus_inverse_b, inverse_std, output_grad
        )
        second = np.einsum(
            "ij, bj, j, bj -> ij",
            centered_input,
            centered_input,
            inverse_std**3,
            output_grad,
        )

        grad_input = first - 1 / B * second

        return grad_input

    def __repr__(self):
        return "BatchNormalization"

In [6]:
class ChannelwiseScaling(Module):
    """
    Implements linear transform of input y = \gamma * x + \beta
    where \gamma, \beta - learnable vectors of length x.shape[-1]
    """

    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1.0 / np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)

        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def _compute_output(self, input):
        output = input * self.gamma + self.beta
        return output

    def _compute_input_grad(self, input, output_grad):
        grad_input = output_grad * self.gamma
        return grad_input

    def _update_parameters_grad(self, input, output_grad):
        self.gradBeta = np.sum(output_grad, axis=0)
        self.gradGamma = np.sum(output_grad * input, axis=0)

    def zero_grad(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)

    def get_parameters(self):
        return [self.gamma, self.beta]

    def get_parameters_grad(self):
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        return "ChannelwiseScaling"

Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 2. Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

Dropout output:

$$f: \mathbb{R}^{B\times N}\rightarrow \mathbb{R}^{B\times N},
f(x) = \cfrac{\text{mask}}{1-p}x,\,f(x)_{bn} = \cfrac{\text{mask}_{bn}}{1-p}x_{bn},\,
\text{mask}_{bn}\sim \text{Bernoulli}(p)$$

Dropout gradient:

$$ \nabla L(x)_{ij} = 
\sum_{b \in 1\dots B, \, n\in 1\dots N}  \cfrac{\partial f_{bn}}{\partial x_{ij}}(x) \nabla \hat L_{bn}=
\sum_{bn}
\cfrac{\text{mask}_{bn}}{1-p}
\cfrac{\partial x_{bn}}{\partial x_{ij}} (x)
\nabla \hat L_{bn}=
\cfrac{\text{mask}_{ij}}{1-p}
\nabla \hat L_{ij}
$$


In [7]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()

        self.p = p
        self.mask = []

    def _compute_output(self, input):
        if self.training == False:
            return input

        self.mask = 1 - np.random.binomial(1, self.p, input.shape)
        output = 1 / (1 - self.p) * self.mask * input

        return output

    def _compute_input_grad(self, input, output_grad):
        grad_input = 1 / (1 - self.p) * self.mask * output_grad

        return grad_input

    def __repr__(self):
        return "Dropout"

## 3. Conv2d

* input: `batch_size x in_channels x h x w`
* output: `batch_size x out_channels x h x w`

You should implement something like pytorch Conv2d layer with `stride=1` and zero-padding outside of image using `scipy.signal.correlate` function.

**Practical notes:**

* While the layer name is "convolution", the most of neural network frameworks (including tensorflow and pytorch) implement operation that is called [correlation](https://en.wikipedia.org/wiki/Cross-correlation#Cross-correlation_of_deterministic_signals) in signal processing theory. **So don't use** `scipy.signal.convolve` since it implements convolution in terms of signal processing.
* It may be convenient to use numpy.pad for zero-padding.
* It's rather ok to implement convolution over 4d array using 2 nested loops: one over batch size dimension and another one over output filters dimension

Conv2d output:

$$f: 
\mathbb{R}^{B\times I\times H \times W}\rightarrow 
\mathbb{R}^{B\times O\times (H + 2P + 1 - K) \times (W + 2P + 1 - K)},
P - \text{padding}, K - \text{kernel size},$$

$$
f(x)_{bohw} = \text{bias}_{o} + \text{conv2d}(x)_{bohw},$$

$$
\text{conv2d}(x;kernel,p)_{bohw} = \sum_{i\in0\dots I-1,\, h_k\in 0\dots K -1,\, w_k \in 0\dots K - 1} 
\text{padded}(x, p)_{b,i,h + h_k, w+w_k} \cdot \text{kernel}_{o i h_k w_k},$$

$$
\text{padded}(x,p)_{b,i,h,w} = [h \in [p, H_x + p - 1] \land w \in [p, W_x + p - 1]] \cdot x_{b, i, h - p, w-p} $$

Conv2d gradient:

$$\nabla L(x)_{b_xi_xh_xw_x} =
\sum_{
b \in 0\dots B-1, \, 
o \in 0\dots O-1,\,
h \in 0\dots H+2P -K,\,
w \in 0\dots W+2P -K
}
\cfrac{\partial f}{\partial x_{b_xi_xh_xw_x}} (x)_{bohw}
\nabla \hat L_{bohw} = \\
$$

$$=
\sum_{b,\,o,\,h,\,w}\,
\sum_{i,\,h_k,\,w_k}
\cfrac{\partial\text{padded}}{\partial x_{b_xi_xh_xw_x}}(x, p)_{b,i,h+h_k,w + w_k} \cdot \text{kernel}_{o i h_k w_k}
\nabla \hat L_{bohw} =
$$

$$=
\sum_{b,\,o,\,h,\,w}\,
\sum_{i,\,h_k,\,w_k}
[h+h_k \in [p, H_x + p - 1] \land w + w_k \in [p, W_x + p - 1]] \cdot 
[b == b_x \land i==i_x \land h + h_k-p == h_x \land w + w_k-p == w_x] \cdot \\
\cdot 
\text{kernel}_{o i h_k w_k}
\nabla \hat L_{bohw}
$$

Let's replace
$$b = b_x,\,i = i_x,\,h = h_x - h_k + p,\,w = w_x - w_k + p: $$

$$\nabla L(x)_{b_xi_xh_xw_x}= 
\sum_{o,\,h_k,\,w_k}
[h_x + p \in [p, H_x + p - 1] \land w_x + p \in [p, W_x + p - 1]] \cdot 
\text{kernel}_{o_x, i, h_k, w_k}
\nabla \hat L_{b,o, h_x - h_k + p, w_x - w_k + p} =\\
=\sum_{o,\,h_k = 0\dots K-1,\,w_k=0\dots K-1}
\text{padded}(\nabla \hat L, p)_{b,o, h_x - h_k + 2p, w_x - w_k + 2p} \cdot 
\text{kernel}_{o_x, i, h_k, w_k}
$$

Notice, that if $K = 2p+1$, then $h'_k \equiv 2p-h_k=K-1 - h_k$ and $w'_k \equiv 2p-w_k=K-1 - w_k$ also take values in the range $0\dots K-1$, and the gradient is equal to:

$$\nabla L(x)_{b_xi_xh_xw_x}= 
\sum_{o,\,h'_k = 0\dots K-1,\,w'_k=0\dots K-1}
\text{padded}(\nabla \hat L, p)_{b,o, h_x + h_k', w_x + w_k'} \cdot 
\text{kernel}_{o_x, i, 2p - h_k', 2p-w_k'}=
\text{conv2d}(\nabla \hat L;kernel',p)_{b_xi_xh_xw_x}$$

Where $kernel'_{oihw} = kernel_{i,o,K -1- h, L-1-w}$, i.e. it is equal to original kernel with transposed first two dimensions and two last dimensions rotated by 180 degrees.

Now let's find the gradient with respect to the kernel:

$$\nabla L_{\text{kernel}}(x)_{o_ki_kh_kw_k} =
\sum_{
b \in 0\dots B-1, \, 
o \in 0\dots O-1,\,
h \in 0\dots H+2P -K,\,
w \in 0\dots W+2P -K
}
\cfrac{\partial f}{\partial \text{kernel}_{o_ki_kh_kw_k}} (x)_{bohw}
\nabla \hat L_{bohw} = \\
$$

$$=
\sum_{b,\,o,\,h,\,w}\,
\sum_{i',\,h_k',\,w_k'}
\text{padded}(x, p)_{b,i',h + h_k', w+w_k'}
\cfrac{\partial\text{kernel}_{o i' h_k' w_k'}}{\partial \text{kernel}_{o_ki_kh_kw_k}} \cdot 
\nabla \hat L_{bohw} =
$$

$$=
\sum_{b,\,o,\,h,\,w}\,
\sum_{i',\,h_k',\,w_k'}
\text{padded}(x, p)_{b,i',h + h_k', w+w_k'}
[o == o_k \land i'==i_k \land h_k' == h_k \land w'_k== w_k]
\nabla \hat L_{bohw}=\\=
\sum_{b,\,h,\,w}
\text{padded}(x, p)_{b,i_k,h + h_k, w+w_k}
\nabla \hat L_{bohw}=
\text{conv2d}^T(x^T;\nabla \hat L^T,p)_{o_ki_kh_kw_k}$$

Where $\text{conv2d}^T, x^T$ and $\nabla \hat L^T$ have their first two dimensions transposed.

And the gradient with respect to the bias:

$$\nabla L_{\text{bias}}(x)_{o} =
\sum_{
b \in 0\dots B-1, \, 
o' \in 0\dots O-1,\,
h \in 0\dots H+2P -K,\,
w \in 0\dots W+2P -K
}
\cfrac{\partial \text{bias}_{o'}}{\partial \text{bias}_{o}} (x)_{bohw}
\nabla \hat L_{bohw} = 
\sum_{
bhw
}
\nabla \hat L_{bohw} 
$$

In [119]:
import scipy as sp
import scipy.signal
import skimage


class Conv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(Conv2d, self).__init__()
        assert kernel_size % 2 == 1, kernel_size

        stdv = 1.0 / np.sqrt(in_channels)
        self.W = np.random.uniform(
            -stdv, stdv, size=(out_channels, in_channels, kernel_size, kernel_size)
        )
        self.b = np.random.uniform(-stdv, stdv, size=(out_channels,))
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    @staticmethod
    def _convolve2d(input, weight, padding=0):
        minibatch, in_channels, iH, iW = input.shape
        out_channels, in_channels_weight, kH, kW = weight.shape

        assert in_channels == in_channels_weight

        output = np.zeros(
            (
                minibatch,
                out_channels,
                iH + 2 * padding + 1 - kH,
                iW + 2 * padding + 1 - kW,
            )
        )

        input = np.pad(input, ((0, 0), (0, 0), (padding, padding), (padding, padding)))

        for out in range(out_channels):
            output[:, out] = sp.signal.correlate(
                input, weight[out, None], mode="valid"
            ).squeeze(1)

        return output

    def _compute_output(self, input):
        self._output = self._convolve2d(input, self.W, padding=self.kernel_size // 2)

        self._output += np.expand_dims(self.b, (0, 2, 3))

        return self._output

    def _compute_input_grad(self, input, gradOutput):
        self._input_grad = self._convolve2d(
            gradOutput,
            np.rot90(self.W.transpose(1, 0, 2, 3), k=2, axes=(2, 3)),
            padding=self.kernel_size // 2,
        )

        return self._input_grad

    def accGradParameters(self, input, gradOutput):
        self.gradW = self._convolve2d(
            input.transpose(1, 0, 2, 3),
            gradOutput.transpose(1, 0, 2, 3),
            padding=self.kernel_size // 2,
        ).transpose(1, 0, 2, 3)

        self.gradb = gradOutput.sum(axis=(0, 2, 3))

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        q = "Conv2d %d -> %d" % (s[1], s[0])
        return q